# IMPORTS

In [2]:
# Import necessary libraries
import cv2
import numpy as np
import threading 
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn.metrics import classification_report


# DATA HANDELING

In [3]:
# data_handling.py
from libs import tf  # Import TensorFlow from libs.py
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_data_generator(directory, target_size=(224, 224), batch_size=32, class_mode='categorical', shuffle=True, augment=False):
    """
    Create and return a data generator using ImageDataGenerator and flow_from_directory.

    Args:
    - directory: Path to the directory containing subdirectories for each class.
    - target_size: Tuple (height, width) to resize the images.
    - batch_size: Number of images per batch.
    - class_mode: 'categorical' (default) for one-hot encoding.
    - shuffle: Whether to shuffle the data (True for training, False for validation/test).
    - augment: Whether to apply data augmentation (True for training, False for validation/test).

    Returns:
    - A data generator instance.
    """
    if augment:
        # Data augmentation for training
        datagen = ImageDataGenerator(
            rescale=1./255,         # Normalize pixel values to [0, 1]
            rotation_range=30,      # Random rotation
            width_shift_range=0.2,  # Horizontal shift
            height_shift_range=0.2, # Vertical shift
            shear_range=0.2,        # Shearing
            zoom_range=0.2,         # Random zoom
            horizontal_flip=True,   # Horizontal flip
            fill_mode='nearest'     # Fill mode for augmented pixels
        )
    else:
        # Only rescale for validation and testing
        datagen = ImageDataGenerator(rescale=1./255)

    # Create the data generator
    generator = datagen.flow_from_directory(
        directory=directory,
        target_size=target_size,   # Resize images to the target size
        batch_size=batch_size,    # Number of images per batch
        class_mode=class_mode,    # One-hot encoding for multi-class classification
        shuffle=shuffle           # Shuffle for training; no shuffle for validation/test
    )
    return generator

def load_data(train_dir, val_dir, test_dir, target_size=(224, 224), batch_size=32):
    """
    Load and return the training, validation, and test data generators.

    Args:
    - train_dir: Path to the training dataset directory.
    - val_dir: Path to the validation dataset directory.
    - test_dir: Path to the test dataset directory.
    - target_size: Tuple (height, width) to resize the images.
    - batch_size: Number of images per batch.

    Returns:
    - A tuple of (train_generator, val_generator, test_generator).
    """
    # Training data generator with augmentation
    train_generator = create_data_generator(
        directory=train_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True,  # Shuffle training data
        augment=True   # Apply augmentation
    )

    # Validation data generator without augmentation
    val_generator = create_data_generator(
        directory=val_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False,  # Do not shuffle validation data
        augment=False   # No augmentation
    )

    # Test data generator without augmentation
    test_generator = create_data_generator(
        directory=test_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False,  # Do not shuffle test data
        augment=False   # No augmentation
    )

    return train_generator, val_generator, test_generator

# If running this script directly, test the data generators
if __name__ == "__main__":
    # Replace these paths with the actual dataset directories
    train_dir = 'faces_folder/train'
    val_dir = 'faces_folder/validation'
    test_dir = 'faces_folder/test'

    # Load data generators
    train_gen, val_gen, test_gen = load_data(train_dir, val_dir, test_dir)

    # Print class indices for verification
    print("Class mappings:", train_gen.class_indices)

    # Print some details about the datasets
    print(f"Training samples: {train_gen.samples}")
    print(f"Validation samples: {val_gen.samples}")
    print(f"Test samples: {test_gen.samples}")

Found 210 images belonging to 3 classes.
Found 45 images belonging to 3 classes.
Found 45 images belonging to 3 classes.
Class mappings: {'Denzel': 0, 'Imaan': 1, 'Sandra': 2}
Training samples: 210
Validation samples: 45
Test samples: 45


# TRAINING

In [4]:
# training.py
from model_arc import model_build
from data_handling import load_data

import pickle

def train_model():
    """
    Train the model with training and validation datasets and evaluate it on the test dataset.
    """
    # Dataset directories
    train_dir = 'faces_folder/train'
    val_dir = 'faces_folder/validation'
    test_dir = 'faces_folder/test'

    # Load data
    train_generator, val_generator, test_generator = load_data(train_dir, val_dir, test_dir)

    # Print class mappings for debugging
    print("Class mappings:", train_generator.class_indices)

    # Build the model
    model = model_build()

    # Train the model
    history = model.fit(
        train_generator,
        epochs=10,
        validation_data=val_generator
    )

    # Save the trained model
    model.save('trained_model.h5')
    print("Model saved to 'trained_model.h5'")

    with open('training_history.pkl', 'wb') as f:
        pickle.dump(history.history, f)
    print("history saved")

    # Evaluate the model
    loss, accuracy = model.evaluate(test_generator)
    print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

    return history

if __name__ == "__main__":
    train_model()


Found 210 images belonging to 3 classes.
Found 45 images belonging to 3 classes.
Found 45 images belonging to 3 classes.
Class mappings: {'Denzel': 0, 'Imaan': 1, 'Sandra': 2}


c:\Users\imaan\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 68s 8s/step - accuracy: 0.5406 - loss: 1.8384 - val_accuracy: 0.6667 - val_loss: 0.8003
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 38s 5s/step - accuracy: 0.8202 - loss: 0.7251 - val_accuracy: 0.9111 - val_loss: 0.3249
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 31s 4s/step - accuracy: 0.9291 - loss: 0.1832 - val_accuracy: 0.9556 - val_loss: 0.1824
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 56s 8s/step - accuracy: 0.9582 - loss: 0.2171 - val_accuracy: 0.9556 - val_loss: 0.0944
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 62s 5s/step - accuracy: 0.9775 - loss: 0.0683 - val_accuracy: 0.9778 - val_loss: 0.1089
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 37s 5s/step - accuracy: 0.9552 - loss: 0.1004 - val_accuracy: 0.9333 - val_loss: 0.1271
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 34s 5s/step - accuracy: 0.9671 - loss: 0.1067 - val_accuracy: 0.9333 - val_loss: 0.1250
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 31s 5s/step - accuracy: 0.9614 - loss: 0.1352 - val_accuracy: 0.9333 - val_loss: 0.1919
Epoch 9/

Model saved to 'trained_model.h5'
history saved
2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.9495 - loss: 0.1601
Test Loss: 0.14100562036037445, Test Accuracy: 0.9555555582046509


# GRAPH

In [5]:
from libs import tf
import matplotlib.pyplot as plt
import pickle
def load_history(historypath):
    with open(historypath, 'rb')  as f:
        history = pickle.load(f)
    
    return history

# Save the model
def create_graph(history):
    

    # Plot training and validation metrics
    accuracy = history.model["accuracy"]
    val_accuracy = history.model["val_accuracy"]
    loss = history.model["loss"]
    val_loss = history.model["val_loss"]
    epochs = range(1, len(accuracy) + 1)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, accuracy, "bo-", label="Training accuracy")
    plt.plot(epochs, val_accuracy, "ro-", label="Validation accuracy")
    plt.title("Training and Validation Accuracy")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, "bo-", label="Training loss")
    plt.plot(epochs, val_loss, "ro-", label="Validation loss")
    plt.title("Training and Validation Loss")
    plt.legend()

    plt.show()

if __name__ == "__main__":
    historypath = 'training_history.pkl'
    history = load_history

    
    create_graph(history)

AttributeError: 'function' object has no attribute 'model'

# WEBCAM

In [ ]:
import cv2
import numpy as np
from libs import tf
from tensorflow.keras.models import load_model
import threading
import time

class WebcamFaceRecognition:
    def __init__(self, model_path, class_labels):
        self.model = load_model(model_path)  # Load the trained model
        self.class_labels = class_labels  # Class labels for prediction

        # Thread-safe variables
        self.frame = None  # Current frame from the webcam
        self.predicted_label = "Loading..."  # Prediction result
        self.running = True  # Flag to control threads

        # Initialize the webcam
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            raise RuntimeError("Error: Unable to access the webcam.")

    def preprocess_frame(self, frame, target_size=(224, 224)):
        """
        Preprocess a single frame for model prediction.
        """
        resized_frame = cv2.resize(frame, target_size)  # Resize to model input size
        normalized_frame = resized_frame / 255.0  # Normalize pixel values
        preprocessed_frame = np.expand_dims(normalized_frame, axis=0)  # Add batch dimension
        return preprocessed_frame

    def capture_frames(self):
        """
        Capture frames from the webcam in a separate thread.
        """
        while self.running:
            ret, frame = self.cap.read()
            if not ret:
                print("Error: Unable to read a frame from the webcam.")
                break
            self.frame = cv2.flip(frame, 1)  # Flip horizontally for a mirror effect
            time.sleep(0.01)  # Small delay to reduce CPU usage

    def predict_frame(self):
        """
        Perform predictions in a separate thread.
        """
        while self.running:
            if self.frame is not None:
                # Preprocess the current frame
                preprocessed_frame = self.preprocess_frame(self.frame)

                # Make predictions
                predictions = self.model.predict(preprocessed_frame)
                predicted_class = np.argmax(predictions[0])  # Get class index
                self.predicted_label = f"{self.class_labels[predicted_class]} ({np.max(predictions[0]) * 100:.2f}%)"
            time.sleep(0.05)  # Adjust sleep for prediction frequency

    def display_feed(self):
        """
        Display the webcam feed with predictions in the main thread.
        """
        while self.running:
            if self.frame is not None:
                # Overlay the prediction result on the frame
                cv2.putText(self.frame, f"Prediction: {self.predicted_label}", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

                # Show the frame
                cv2.imshow("Webcam Face Recognition", self.frame)

            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                self.running = False

        # Release resources
        self.cap.release()
        cv2.destroyAllWindows()

    def start(self):
        """
        Start the webcam face recognition with multithreading.
        """
        # Start threads for frame capture and prediction
        capture_thread = threading.Thread(target=self.capture_frames)
        prediction_thread = threading.Thread(target=self.predict_frame)

        capture_thread.start()
        prediction_thread.start()

        # Run the display in the main thread
        self.display_feed()

        # Wait for threads to finish
        capture_thread.join()
        prediction_thread.join()

if __name__ == "__main__":
    # Path to the trained model
    model_path = "trained_model.h5"

    # List of class labels (must match the training labels)
    class_labels = ["Imaan", "Denzel", "Sandra"]

    # Initialize and run the face recognition
    recognition_system = WebcamFaceRecognition(model_path, class_labels)
    recognition_system.start()
